<a href="https://colab.research.google.com/github/MeeTK26/30DaysOfMLDL/blob/main/HouseHoldPowerConsumptionUsingRNN(day28).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
import tensorflow as tf

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00235/household_power_consumption.zip"
data_path = tf.keras.utils.get_file("household_power.zip", url, extract=True)
data_file = data_path.replace(".zip", "") + "/household_power_consumption.txt"

19759104/Unknown 2s 0us/step

In [3]:
df = pd.read_csv(data_file, sep=";", parse_dates={"datetime": ["Date", "Time"]}, infer_datetime_format=True,
                 na_values=["?"], low_memory=False)
df.fillna(method="ffill", inplace=True)
df["Global_active_power"] = df["Global_active_power"].astype(float)
df.set_index("datetime", inplace=True)

<ipython-input-3-c01e95fe4c82>:1: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(data_file, sep=";", parse_dates={"datetime": ["Date", "Time"]}, infer_datetime_format=True,
<ipython-input-3-c01e95fe4c82>:1: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv(data_file, sep=";", parse_dates={"datetime": ["Date", "Time"]}, infer_datetime_format=True,
<ipython-input-3-c01e95fe4c82>:1: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df = pd.read_csv(data_file, sep=";", parse_dates={"datetime": ["Date", "Time

In [4]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df[["Global_active_power"]])
sequence_length = 48

In [5]:
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

In [6]:
from sklearn.model_selection import train_test_split

X, y = create_sequences(df_scaled, sequence_length)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN,  Dense

rnn_model=Sequential([
        SimpleRNN(50, activation="relu", return_sequences=False, input_shape=(sequence_length, 1)),
        Dense(1)
    ])
rnn_model.compile(optimizer="adam", loss="mse")

In [11]:
rnn_history = rnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=1)


Epoch 1/10
51881/51881 ━━━━━━━━━━━━━━━━━━━━ 257s 5ms/step - loss: 6.0984e-04 - val_loss: 3.6602e-04
Epoch 2/10
51881/51881 ━━━━━━━━━━━━━━━━━━━━ 264s 5ms/step - loss: 5.6810e-04 - val_loss: 3.5503e-04
Epoch 3/10
51881/51881 ━━━━━━━━━━━━━━━━━━━━ 322s 5ms/step - loss: 5.6399e-04 - val_loss: 3.5386e-04
Epoch 4/10
51881/51881 ━━━━━━━━━━━━━━━━━━━━ 323s 5ms/step - loss: 5.5993e-04 - val_loss: 3.5736e-04
Epoch 5/10
51881/51881 ━━━━━━━━━━━━━━━━━━━━ 313s 5ms/step - loss: 5.5547e-04 - val_loss: 3.6716e-04
Epoch 6/10
51881/51881 ━━━━━━━━━━━━━━━━━━━━ 271s 5ms/step - loss: 5.7288e-04 - val_loss: 3.7747e-04
Epoch 7/10
51881/51881 ━━━━━━━━━━━━━━━━━━━━ 310s 5ms/step - loss: 5.8563e-04 - val_loss: 3.7500e-04
Epoch 8/10
51881/51881 ━━━━━━━━━━━━━━━━━━━━ 265s 5ms/step - loss: 5.6620e-04 - val_loss: 3.9489e-04
Epoch 9/10
51881/51881 ━━━━━━━━━━━━━━━━━━━━ 260s 5ms/step - loss: 5.5681e-04 - val_loss: 3.4605e-04
Epoch 10/10
51881/51881 ━━━━━━━━━━━━━━━━━━━━ 262s 5ms/step - loss: 5.5555e-04 - val_loss: 3.6736e-04